In [4]:
import numpy as np
import matplotlib.pyplot as plt

In [8]:
def FNS(scores):
    domination = (scores[:, None, :] <= scores[None, :, :]).all(2) # domination [i, j] = "i dominuje j"
    domination &= ~(scores[:, None, :] == scores[None, :, :]).all(2)
    Nx = domination.sum(0)
    
    Pf = []
    ranks = np.zeros(scores.shape[0])
    r = 0
    Q = np.argwhere(Nx == 0)
    while Q.size > 0:
        Q = Q[:, 0]
        Nx[Q] = -1
        Pf.append(Q)
        ranks[Q] = r
        r += 1
        for i in Q:
            Nx[domination[i, :]] -= 1
        Q = np.argwhere(Nx == 0)
        
    return Pf, ranks

def crowding_distance(scores):
    indices = np.argsort(scores, 0)
    sorted_scores = np.take_along_axis(scores, indices, 0)
    cd = np.zeros(scores.shape[0])
    for k in range(scores.shape[1]):
        cd[indices[[0, -1], k]] = np.inf
        cd[indices[1:-1, k]] += (sorted_scores[2:, k] - sorted_scores[:-2, k]) / (sorted_scores[-1, k] - sorted_scores[0, k])
    return cd

def random_population(d, n, x_min, x_max):
    return np.hstack([np.random.uniform(x_min, x_max, (n, d))])

def tournament_selection(ranks, dists, n):
    candidates = np.random.choice(n, (n, 2), replace=True)
    mask = np.where(
        ranks[candidates[:, 0]] == ranks[candidates[:, 1]],
        dists[candidates[:, 0]] > dists[candidates[:, 1]],
        ranks[candidates[:, 0]] < ranks[candidates[:, 1]]
    )
    result = candidates[:, 1]
    result[mask] = candidates[mask, 0]
    return result

def crossover(x, eta): # simulated binary crossover
    n, d = x.shape
    l = n // 2
    mi = np.random.random((l, d))
    beta = np.where(
        mi < 0.5,
        np.power(2*mi, 1. / (eta+1.)),
        np.power(1. / (2.*(1-mi)), 1. / (eta+1.))
    )
    c1 = 0.5 * (1 + beta) * x[:l, :] + 0.5 * (1 - beta) * x[l:, :]
    c2 = 0.5 * (1 + beta) * x[:l, :] + 0.5 * (1 - beta) * x[l:, :]
    return np.vstack([c1, c2])

def elitist_selection(fronts, dists, to_take):
    taken = []
    for front in fronts:
        if len(front) <= to_take:
            taken += list(front)
            if len(front) == to_take:
                break
            to_take -= len(front)
        else:
            indices = np.argsort(dists[front])[:to_take]
            taken += list(indices)
            break
    return taken

def IDEA(objective, constraint, x_min, x_max, d, n, n_inf, eta_c, eta_m, p_mut, num_iterations, log_interval=10): # TODO
    n_f = n - n_inf
    population = random_population(d, n, x_min, x_max)
    populations = [population.copy()]
    scores = np.stack([objective(population), constraint(population)], 1)
    scores_hist = [scores.copy()]

    fronts, ranks = FNS(scores)
    dists = crowding_distance(scores)
    
    for iter_ in range(num_iterations):
        parent_indices = tournament_selection(ranks, dists, n)
        offspring = crossover(population[parent_indices, :], eta_c)
        offspring = np.clip(offspring, x_min, x_max)
        offspring = mutation(offspring, x_min, x_max, p_mut, eta_m)
        population = np.vstack([population, offspring])
        
        scores = np.stack([objective(population), constraint(population)], 1)
        dists = crowding_distance(scores)
        mask_f = scores[:, -1] == 0
        mask_inf = ~mask_f
        s_f = np.sum(mask_f)
        s_inf = np.sum(mask_inf)
        if s_f < n_f:
            to_take_f = s_f
            to_take_inf = n - s_f
        elif s_inf < n_inf:
            to_take_inf = s_inf
            to_take_f = n - s_inf
        else:
            to_take_f = n_f
            to_take_inf = n_inf
            
        fronts, ranks = FNS(population[mask_f])
        taken_f = elitist_selection(fronts, dists[mask_f, :], to_take_f)
        fronts, ranks = FNS(population[mask_f])
        taken_inf = elitist_selection(fronts, dists[mask_inf, :], to_take_inf)
        
        indices_f = np.argwhere(mask_f)[:, 0][taken_f]
        indices_inf = np.argwhere(mask_inf)[:, 0][taken_inf]
        indices = np.hstack([indices_f, indices_inf])
        
        population = population[indices, :]
        scores = scores[indices, :]
        fronts, ranks = FNS(population)
        dists = dists[taken]
        populations.append(population.copy())
        scores_hist.append(scores.copy())
        
        if iter_ % log_interval == 0:
            print(f"Iteration {iter_}, scores: {scores.min(0)} {scores.mean(0)} {scores.max(0)}")
    print(f"Iteration {iter_}, scores: {scores.min(0)} {scores.mean(0)} {scores.max(0)}")
    return np.stack(populations, 0), np.stack(scores_hist, 0)